In [ ]:
import xarray as xr
import numpy as np
import hydromt
from os.path import join, basename, dirname
import matplotlib.pyplot as plt
import matplotlib as mpl
from hydromt.models import SfincsModel
import subprocess
import os

In [ ]:
from hydromt_sfincs.workflows import burn_bathymetry, get_river_bathymetry, get_estuary_bathymetry
from hydromt_sfincs.utils import write_binary_map, write_inp
from hydromt_sfincs.gis_utils import flipud
import pyflwdir

In [ ]:
fn_exe = "p:/11205283-hydromt-floodmodelling/02_models/bin/subgrid_openacc_11_rev295_16092021/sfincs.exe"
mdir = r"../02_models/beira/sfincs"

In [ ]:
mod00 = SfincsModel(join(mdir, '00_base'), mode='r')
mod00.read()

In [ ]:
mod01 = SfincsModel(join(mdir, '01_base_full'), mode='r', deltares_data=True, data_libs=join(mdir, 'data_sources.yml'))
mod01.read()

## create restart file run with average conditions

In [ ]:
import pandas as pd
q = xr.open_dataarray(join(mdir, 'forcing_data',  'cama_outflw_era5.nc'))
qmean = q.mean('time').expand_dims('time').transpose('index', 'time')
qmean['time'] = xr.IndexVariable('time', pd.to_datetime(['20200101']))
dis = qmean.reindex_like(mod01.forcing['dis'], method='nearest')
havg = mod01.forcing['bzs'].mean().expand_dims('time').expand_dims('index')
havg['time'] = xr.IndexVariable('time', pd.to_datetime(['20200101']))
havg['index'] = xr.IndexVariable('index', [0])
havg = havg.reindex_like(mod01.forcing['bzs'], method='nearest')
havg['x'] = mod01.forcing['bzs']['x']
havg['y'] = mod01.forcing['bzs']['y']

In [ ]:
root = basename(mod01.root)
config = mod01.config.copy()
for k, v in config.items():
    if k[:-4] in ['dis', 'bzs']:
        config.update({k: basename(v)})
    elif k.endswith('file') and '01_base_full' not in v:
        config.update({k:f'../{root}/{basename(v)}'})
config.update({'alpha': 0.7})
config.update({'restartfile': 'sfincs.restart'})
tref = config['tref']

# no precip
_ = config.pop('netampr', None)
_ = mod01.forcing.pop('netampr', None)

# mean dis, bzs
mod01.set_forcing(dis, name='dis')
mod01.set_forcing(havg, name='bzs')

mod01.set_root(join(mdir, '01_base_full_mean'), mode='w')
mod01.config.update(**config)
mod01.write_forcing()
mod01.write_config()

## rivdph runs

In [ ]:
river_upa = 25

da_msk0 = mod01.staticmaps['msk'].copy()
# da_scs0 = mod01.staticmaps['scs'].copy()

ds_mod0 = flipud(mod00.staticmaps)
ds_mod1 = flipud(mod01.staticmaps)
da_elv0 = ds_mod0['dep'].copy()
da_upa0 = ds_mod1["uparea"].copy()
da_flw0 = ds_mod1["flwdir"].copy()

rivd8 = da_upa0 > river_upa
flwdir = hydromt.flw.flwdir_from_da(da_flw0, mask=rivd8)

# geoms
gdf_stream = mod01.staticgeoms['rivers'].copy()
# flw = pyflwdir.from_dataframe(gdf_stream.set_index("idx"))
# flw.main_upstream(gdf_stream["uparea"].values)
# rivqbf = np.maximum(0, gdf_stream["qbankfull"].fillna(0))
# estuary = gdf_stream["estuary"].values

In [ ]:
da_rivmask = mod01.data_catalog.get_rasterdataset('grwl_mask', geom=mod01.region, buffer=10)
_mask_dst = da_elv0.raster.reproject_like(da_rivmask) != da_elv0.raster.nodata
da_rivmask = da_rivmask.where(_mask_dst, 0) > 0

In [ ]:
hmin=1.0

root = basename(mod01.root)
config = mod01.config.copy()
for k, v in config.items():
    if k.startswith('dep'):
        config.update({k:'sfincs.dep'})
    elif k.endswith('file') and not '01_base_full' in v:
        config.update({k:f'../{root}/{basename(v)}'})
config.update({'dtout': 86400})
config.update({'alpha': 0.7})

hps = np.arange(0.25, 0.41, 0.05)
hcs = np.arange(0.25, 0.41, 0.05)
# hcs=[0.27]
# hps=[0.30]

irun = 10
for hc, hp in zip(hcs, hps):
# for hc, hp in zip([0.35], [0.30]):
    irun += 1
    gdf_stream = mod01.staticgeoms['rivers'].copy().drop(columns=['rivdph'])
    gdf_stream = get_river_bathymetry(gdf_stream, hc=hc, hp=hp)
    da_estuary=None
    gdf_stream, da_estuary = get_estuary_bathymetry(gdf_stream, da_rivmask=da_rivmask)
    da_elv = burn_bathymetry(gdf_stream, da_elv0, da_rivmask=da_estuary)[0]
    da_elv.data = flwdir.dem_dig_d4(da_elv.values)
    
    davg = gdf_stream['rivdph'].mean()
    dmax = gdf_stream['rivdph'].max()

    run = f'02_{irun:02d}_hc{hc*100:02.0f}_hp{hp*100:02.0f}_D4'
    print(f'{run} mean {davg:.2f} max {dmax:.2f}')
    
    root = join(mdir, run)
    if not os.path.isdir(root):
        os.makedirs(root)
    write_binary_map(join(root, 'sfincs.dep'), flipud(da_elv).values, da_msk0.values)
    write_inp(join(root, 'sfincs.inp'), config)


In [ ]:
mod = SfincsModel(join(mdir, run), mode='r')
mod.read()
mod.plot_basemap(variable='dep', geoms=['bnd', 'src'])

## scs runs

(not used with ERA5 runoff based 'net precip')

## runoff runs

In [ ]:
mod01.data_catalog.from_yml(r'../02_models/beira/cmf/out/cama.yml')

In [ ]:
precip = mod01.forcing['netampr'].copy()
_ = mod01.forcing.pop('bzs', None)

root = basename(mod01.root)
config = mod01.config.copy()
for k, v in config.items():
    if k.startswith('dis') or k.startswith('netampr'):
        config.update({k:basename(v)})
    elif k.endswith('file') and not '01_base_full' in v:
        config.update({k:f'../{root}/{basename(v)}'})
config.update({'dtout': 86400})
config.update({'alpha': 0.7})
mod01.config.update(**config)

In [ ]:
mod01.set_root('../02_models/beira/sfincs/02_051_ro080', mode='w')
mod01.setup_q_forcing_from_grid(
    discharge_fn='cama_outflw080',
    uparea_fn='cama_uparea',
)
mod01.set_forcing(precip*0.8, name='netampr')
mod01.write_forcing()
mod01.write_config()
# mod01.plot_forcing()

In [ ]:
mod01.set_root('../02_models/beira/sfincs/02_061_ro120', mode='w')
mod01.setup_q_forcing_from_grid(
    discharge_fn='cama_outflw120',
    uparea_fn='cama_uparea',
)
mod01.set_forcing(precip*1.2, name='netampr')
mod01.write_forcing()
mod01.write_config()
# mod01.plot_forcing()

## compound runs

In [ ]:
# get htide
mod01.setup_h_forcing(
    geodataset_fn = 'idai_tides_his_beira_egm',   # waterlevel timeseries dataset
    buffer = 1000
)
mod01.forcing['bzs'].attrs.pop('category', None)
mod01.forcing['bzs'].reset_coords(drop=True).to_netcdf(join(mdir, 'forcing_data', 'htide_idai.nc'))

### RE-INITIALIZE mod01 above

In [ ]:
# get qmean
# mod01.setup_config(**dict(
#     tref = '19800101 000000',
#     tstart = '19800101 000000',
#     tstop = '20191231 000000',
# ))
# mod01.setup_q_forcing_from_grid(
#     discharge_fn = 'cama_outflw',
#     uparea_fn = 'cama_uparea',
#     rel_error = 0.05,
#     abs_error = 100
# )
# mod01.forcing['dis'].load()
# mod01.forcing['dis'].to_netcdf(join(mdir, 'forcing_data',  'cama_outflw_era5.nc'))
# mod01.read()
qts = mod01.forcing['dis'].copy()
q = xr.open_dataarray(join(mdir, 'forcing_data',  'cama_outflw_era5.nc'))
qmean = q.groupby('time.month').mean().sel(month=int(np.mean(qts.time.dt.month)))
fact = np.minimum(1,qmean/qts.mean())
print(fact.values)
da_disclim = qts*fact

htide = xr.open_dataarray(join(mdir, 'forcing_data',  'htide_eloise.nc'))

In [ ]:
from itertools import product
from hydromt_sfincs.utils import write_timeseries

root = basename(mod01.root)
config = mod01.config.copy()
for k, v in config.items():
    if k[:-4] in ['dis', 'bzs', 'netampr']:
        config.update({k: basename(v)})
    elif k.endswith('file') and '01_base_full' not in v:
        config.update({k:f'../{root}/{basename(v)}'})
config.update({'alpha': 0.7})
tref = config['tref']

qs = {
    'dis': mod01.forcing['dis'].to_series().unstack(0), 
    'disclim': da_disclim.to_series().unstack(0)
}
hs = {
    'htot': mod01.forcing['bzs'].to_series().unstack(0),
    'htide': htide.to_series().unstack(0),
}
ps = {
    'precip': mod01.forcing['netampr'].copy(),
#     'runoff': 
    'noprecip': None
}

irun = 10
# combis = [
#     ('dis', 'htot', 'precip'), 
#     ('dis', 'htide', 'noprecip'),
#     ('disclim', 'htot', 'noprecip'),
#     ('disclim', 'htide', 'precip'),
# ]:

for q, h, p in product(qs.keys(), hs.keys(), ps.keys()):
    irun += 1
    
    config1 = config.copy()
    
    run = f'03_{irun:02d}_{h}_{q}_{p}'
    root = join(mdir, run)
    if not os.path.isdir(root):
        os.makedirs(root)

    write_timeseries(join(root, 'sfincs.dis'), qs[q], tref)
    write_timeseries(join(root, 'sfincs.bzs'), hs[h], tref)
    if ps[p] is None:
        config1.pop('netamprfile')
    else:
        ps[p].to_netcdf(join(root, 'precip.nc'))
        
    write_inp(join(root, 'sfincs.inp'), config1)
    
#     mod = SfincsModel(join(mdir, run), mode='r')
#     mod.plot_forcing()
#     plt.close()
#     if 'precip' in mod.forcing:
#         mod.forcing['precip'].close()

## do runs

In [ ]:
import glob
from os.path import join, basename
import os
import subprocess

runs = glob.glob(join(mdir, '02_*'))# + glob.glob(join(mdir, '03_*'))
# runs = glob.glob(join(mdir, '01_*mean*'))
len(runs), runs

In [ ]:
import subprocess

for root in runs:
#     if os.path.isfile(os.path.join(root, 'sfincs_map.nc')):
#         continue
    print(os.path.basename(root))
    with open(os.path.join(root, "sfincs.log"), 'w') as f:
        p = subprocess.Popen([fn_exe], stdout=f, cwd=root)
        p.wait()